In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sys 
sys.path.append('../maths/') 
from baseball_stats import BasicHitting


In [2]:
k = ['playerID', 'birthYear', 'birthMonth', 'birthDay']
import_people = pd.read_csv('../datafiles/People.csv', encoding='latin-1', usecols=k)
import_batting = pd.read_csv('../datafiles/Batting.csv', encoding='latin-1')
people = import_people.copy()

#fix fucky dates
people['birthYear'] = people['birthYear'].fillna(1875).astype(int)
people['birthMonth'] = people['birthMonth'].fillna(1).astype(int)
people['birthDay'] = people['birthDay'].fillna(1).astype(int)

#create a birthdate column so we can calulate an age at start of season
people['birthdate'] = people.apply(lambda x: f"""{x['birthYear']}-{x['birthMonth']}-{x['birthDay']}""", axis=1)
people['birthdate'] = pd.to_datetime(people['birthdate'], errors='coerce')

#merge the people and batting dataframes
batting = people.merge(import_batting, on='playerID', how ='inner')


In [ ]:
#TODO: create as function to do career and either 162 game or 600 pa stats
season_batting = batting.copy()
season_batting = season_batting.drop(columns=['stint', 'teamID', 'lgID', 'birthYear', 'birthMonth', 'birthDay', 'G_old', 'G_batting']).groupby(['playerID', 'birthdate', 'yearID']).sum().reset_index()
#estimate the start of the season at april 1
season_batting['season_start'] = season_batting.apply(lambda x: f"""{x['yearID']}-04-01""", axis=1)
season_batting['season_start'] = pd.to_datetime(season_batting['season_start'], errors='coerce')

#calculate the age of the player at the start of the season
season_batting['age'] = season_batting.apply(lambda x: relativedelta(x['season_start'], x['birthdate']).years, axis=1)

season_batting.rename(columns={'3B':'B3', '2B':'B2'}, inplace=True)

#adding shit up
season_batting['PA'] = BasicHitting.calc_pa(season_batting)
season_batting['B1'] = BasicHitting.calc_b1(season_batting)
season_batting['TB'] = BasicHitting.calc_tb(season_batting)
season_batting['SBA'] = BasicHitting.calc_sba(season_batting)
season_batting['BA'] = BasicHitting.calc_ba(season_batting)
season_batting['XBH'] = BasicHitting.calc_xbh(season_batting)
season_batting['TOB'] = BasicHitting.calc_tob(season_batting)
season_batting['RP'] = BasicHitting.calc_rp(season_batting)

#ratios and percentages
season_batting['HRAB'] = BasicHitting.calc_hrab(season_batting)
season_batting['HRH'] = BasicHitting.calc_hrh(season_batting)
season_batting['OBP'] = BasicHitting.calc_obp(season_batting)
season_batting['SLG'] = BasicHitting.calc_slg(season_batting)
season_batting['OPS'] = BasicHitting.calc_ops(season_batting)
season_batting['EQA'] = BasicHitting.calc_eqa(season_batting)
season_batting['ISO'] = BasicHitting.calc_iso(season_batting)
season_batting['GPA'] = BasicHitting.calc_gpa(season_batting)
season_batting['RC'] = BasicHitting.calc_rc(season_batting)
season_batting['BR'] = BasicHitting.calc_br(season_batting)
season_batting['SBP'] = BasicHitting.calc_sbp(season_batting)
season_batting['TA'] = BasicHitting.calc_ta(season_batting)
season_batting['BABIP'] = BasicHitting.calc_babip(season_batting)

#TODO:pa per so
#TODO:bb per strikeout
#TODO:extra base hit pct



season_batting

AttributeError: type object 'BasicHitting' has no attribute 'calc'

In [ ]:
#TODO: combine and export to csv